# Assignment 1: Unit tests and coverage

- [1. Introduction](#1.-Introduction)
- [2. Coverage](#2.-Coverage)
    - [2.1 Statement coverage](#2.1-Statement-coverage)
    - [2.2 Branch coverage](#2.2-Branch-coverage)
    - [2.3 Dataflow coverage](#2.3-Dataflow-coverage)
- [3. More unit tests](#3.-More-unit-tests)
- [4. Mocking](#4.-Mocking)
- [5. Coverage revisited](#5.-Coverage-revisited)
- [BONUS: `doctest`](#BONUS:-doctest)
- [6. Submit to Canvas](#6.-Submit-to-Canvas)

## 1. Introduction

For a new self-driving car, we need an implementation of a high-precision pi: ChatGPT v4 suggests the following implementation for computing pi in Python, including a unit test. The code is packed in the two files `estimate_pi.py` and `test_estimate_pi.py`. 

Run the existing test using your shell (every cell starting with an `!` will be executed in your OS's shell). 

In [18]:
!python3 test_estimate_pi.py

.
----------------------------------------------------------------------
Ran 1 test in 0.389s

OK


What is problematic with that test ChatGPT created for us and would you address this problem?

### Problems:
- The test does not test wether or not the input parameter, **n** is an integer thats greater than 0. We need to test this otherwise it will result in **division by zero**. I would adress this problem by implementing another class method that checks if the input is greater than 0.
- There is no test case for testing if the input is of type **int**, meaning we might enter "100" instead of 100. I would adress this problem by implementing another class method that checks for datatype of the input.
- Since this estimation of pi is quite random, the tests should be executed multiple times to ensure expected functionallity.
- The **delta** parameter might lead in an un precice result which then leads to unexpected returnvalues.

## 2. Coverage

### 2.1 Statement coverage
Compute the statement coverage of the program using [`coverage.py`](https://coverage.readthedocs.io/en/latest/index.html). 

In [19]:
!coverage run test_estimate_pi.py
!coverage report

.
----------------------------------------------------------------------
Ran 1 test in 0.863s

OK
Name                  Stmts   Miss  Cover
-----------------------------------------
estimate_pi.py           26     12    54%
test_estimate_pi.py      11      0   100%
-----------------------------------------
TOTAL                    37     12    68%


How can we interprete the results?

The way i think of this result is that in the **test_estimate_pi.py file**, we cover 54% of the code from **estimate_pi.py**. Meaning we test 54 % of it. The missing column tells us what we are not testing, which makes sense if we look at the code. Line 18-19 is where we open the file in **estimate_pi.py** since we are mocking in the **test_estimate_pi.py** we do not really open any files, and i assume thats what it means.

The most important thing to remember when interpreting coverage is that even if the test results in 100% coverage does not that automatically mean that the code is bug free. Coverage simply returns an indication of how **complete** the tests are. So in this case the code generated by chatGPT is actually covering 100% of the code, but i found several flaws that would lead to unexpected performance.

### 2.2 Branch coverage
Now compute the statement coverage of the program using [`coverage.py`](https://coverage.readthedocs.io/en/latest/index.html). 

In [24]:
!coverage run --branch test_estimate_pi.py
!coverage report -m

.
----------------------------------------------------------------------
Ran 1 test in 0.937s

OK
Name                  Stmts   Miss Branch BrPart  Cover   Missing
-----------------------------------------------------------------
estimate_pi.py           26     12     12      1    50%   18-19, 23-38
test_estimate_pi.py      11      0      2      1    92%   15->exit
-----------------------------------------------------------------
TOTAL                    37     12     14      2    61%


How can we interprete the results?

We can interpret the statement coverage report like we did with the cover report. It tells us helps us understand how much of the code written in **estimate_pi.py** in terms of statements(i.e functioncalls, ifstatements, whileloops) are actually being executed. This gives me an idea on how i can construct new tests to test my code further and to ensure that every part of the code is actually tested. 

In the output above, we can see that we cover **92%** of the statements in **test_estimate_pi.py**. The reason for getting **92 %** coverage is that we are not testing if the name is **main** anywhere. We cover **50 %** of the code in **estimate_pi**, this percentage seems fair since we are only testing  one specific function, the rest is without any unittests. If we look at the **Missing** column, we can see that more clearly.

### 2.3 Dataflow coverage

Draw the flow graph for the function `estimate_pi` defined in `estimate_pi.py`. Annotate the graph with definition and use information. Note: Please submit a separate image file or PDF with the name `dataflow_coverage.<file_extension>` for this task.

Identify and describe the minimum number of test cases to achieve: all-defs coverage, and all-uses coverage. 

### All-defs coverage
For all defs coverage we need to make sure that all the variables are defined at least once. The variables that should be checked for is **n**, **count**, **x**, and **y**. The way im going to structure this is by dividing it into different cases:

**Case 1** - minimum n = 1  
If the input parameter, **n** is 1, then i expect an output in the range of [0, 4]. By having n = 1 we make sure that we go through each branch once. 

### All-uses coverage
For the all uses coverage we need to make sure that every node and edge is visited, meaning we need to ensure that every possible combination of nodes is visited. Im going to divide them up into multiple testcases again. For all the testcases i assume n to be greater than 0, otherwise we will not achieve anything:  

**Case 1** - True branch   
If the input parameter, **n**, is lets say 100 the expected output is a somewhat good estimation of pi. 

**Case 2** - False branch
If the input parameter, **n**, is lets say 1, there is a possibility that the randomized values of **x** and **y** is going to result in a value greater than 1, meaning we enter the false branch meaning our expectations of the function is not met.

### Summary
Although we can define some test cases, it is worth keeping in mind that this function is not deterministic due to its randomness. Meaning the expected output may vary within an acceptable range of expected outputs.


## 3. More unit tests

Add two more unit tests with the principles you learned in the lecture. Describe what principle you have used.

I follow each and every of the principles, below is a brief explanation to why:
- Fast: The code is fast due to simple checks of values, there is no fancy stuff happening.
- Independend: There is no certain order that the code or tests need to be run in for it to work as intended
- Repeatable: The code does not depend on any extern environment/instance and since certain pre-defined values for **n** that is making sure that every branch and edge is visited. The code is therefore repeatable
- Self-validating: The code is validating itself by using the build in assert methods.
- Thorough: We cover each case due to the pre-defined values of **n**. 

## 4. Mocking

We want to store the resulting number persistently on our file system. We use the following class. 

In [25]:
class PiFileWriter:
    @staticmethod
    def write(content, file_path):
        with open(file_path, 'w') as file:
            file.write(content)

Implement a test double for `PiFileWriter` and add your implementation to `test_estimate_pi.py`. Discuss what type of test double you have implemented.

I have implemented another class for testing **PiWriter**. The class contains one method:  
1. test_file_writer: This methods mocks the build in open method in Python, simulating a file open without actually opening or writing anything. It also chekcs so that **with open** is called with correct parameters aswell as checking if its only called once.

This method is of type **Mock** since it simulates the behaviour of a file being opened.

In [25]:
!python3 test_estimate_pi.py

......
----------------------------------------------------------------------
Ran 6 tests in 0.411s

OK


Name three other types of unit tests you would want to mock and explain why. 

I kind of already implemented two of the three im going to talk about, but not exacly with mocking. Here are three more unittests that takes advantage of mocks and spies:
1. **Mocking** random.uniform and making sure it returns the same value every time. For one instance i could mock a value to be greater than 1, meaning the the result would be outside the unit circle.
2. **Mocking** random.uniform again, but making sure it returns a value less than one which leads to the result being within the unitcircle.
3. Using a **Spy** to make sure that the function is called with the correct arguments. For instance implementing functionallity that only looks/records the calls of the **write** function making sure that the arguments are of the correct datatype and is in the correct order. This spy would then use **mocking** when opening the file, simulating it.

## 5. Coverage revisited

Rerun statement and branch coverage and discuss the differences and changes.

In [26]:
!coverage run test_estimate_pi.py
!coverage report

......
----------------------------------------------------------------------
Ran 6 tests in 0.904s

OK
Name                  Stmts   Miss  Cover
-----------------------------------------
estimate_pi.py           26     10    62%
test_estimate_pi.py      34      0   100%
-----------------------------------------
TOTAL                    60     10    83%


In [27]:
!coverage run --branch test_estimate_pi.py
!coverage report -m

......
----------------------------------------------------------------------
Ran 6 tests in 0.948s

OK
Name                  Stmts   Miss Branch BrPart  Cover   Missing
-----------------------------------------------------------------
estimate_pi.py           26     10     12      1    61%   23-38
test_estimate_pi.py      34      0      8      1    98%   57->exit
-----------------------------------------------------------------
TOTAL                    60     10     20      2    80%


We can see that the **TOTAL** coverage increased from **61 %** to **80 %**. The coverage in **estimate_pi** increased from **50 %** to **61 %** and from **92 %** to **98 %** in **test_estimate_pi**. I would say that this is a significant increase by adding some more unittests. 

When i previously ran the coverage tests, i was not testing line 18-19, being where we write to a file. However now that i execute the tests once again, those lines are gone meaning those lines are being tested. 

### Summary
In summary, the code thats either estimating pi or writing to file is now being tested. Line 23-38 is still not being tested, however this task is about **unit tests** meaning im not going to test all of the code. Im happy that i managed to write tests that seems to be working as intended.

# BONUS: `doctest`

If you are curious or want to stand out, check out [`doctest`](https://en.wikipedia.org/wiki/Doctest). This task is optional. 

Add two `doctest` test cases and run the `doctest` tests.

In [ ]:
!

How do you like `doctest`?

## 6. Submit to Canvas

Almost done, but the most tricky part is missing: submitting. :)

Before submitting, make sure
- you completed all non-optional tasks in this assignment (i.e., all empty cells are filled with meaningful content)
- you don't use external libraries except `coverage.py`
- the notebook runs straight through
- your test code works
- your code is readable and follows the Python coding conventions

All set? Great. Just two steps away from happiness. 

1. Go through the list above and check again
2. Submit *three* files to canvas:
    - `assignment.ipynb`
    - `test_estimate_pi.py`
    - `dataflow_coverage.<file_extension>`
3. Take a deep breath and carpe diem.
